In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
sys.setrecursionlimit(5000)

In [3]:
from filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
    get_unique_nutzung
)

from compute_ET_green import compute_et_green, calculate_band_std_dev
from compute_ET_blue import (
    compute_et_blue,
    compute_volumetric_et_blue,
    postprocess_et_blue,
)
from add_walter_irrigation_estimates import filter_and_estimate_irrigation

from utils import ee_utils, date_utils
from utils.ee_utils import back_to_float, back_to_int, export_image_to_asset

from typing import List, Tuple

import ee
import geemap

In [ ]:
ee.Initialize(project="thurgau-irrigation")

## Compute and export ET Green

In [ ]:
TIME_STEPS = 36
TIME_STEP_TYPE = "dekadal"
YEARS = range(2018, 2024)

In [ ]:
DOUBLE_CROPPING_IMAGE = ee.Image(
    "projects/thurgau-irrigation/assets/Zuerich/crop_vegetation_period_zh_2022"
)

In [16]:
path_to_WaPOR_ET_asset = "projects/thurgau-irrigation/assets/Thurgau/ET_WaPOR_10m_dekadal_test_refactor"


WaPOR_ET_collections = ee_utils.merge_collections(
    years=YEARS, asset_name=path_to_WaPOR_ET_asset
)

WaPOR_ET_collections = WaPOR_ET_collections.map(lambda image: back_to_float(image, 100))

10

In [20]:
Map = geemap.Map()

image_2022 = ee.ImageCollection("projects/thurgau-irrigation/assets/Thurgau/ET_WaPOR_10m_dekadal_test_refactor_2022").first()
image_2023 = ee.ImageCollection("projects/thurgau-irrigation/assets/Thurgau/ET_WaPOR_10m_dekadal_test_refactor_2023").first()

vis_params = {
    "bands": ["downscaled"],
    "min": 0,
    "max": 0.01,
    "palette": [
        "000000",
        "005aff",
        "43c8f7",
        "fff83d",
        "ff0000",
        "ff00ff",
    ],
}

Map.addLayer(image_2022, vis_params, "ET 2022")
Map.addLayer(image_2023, vis_params, "ET 2023")

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
PATH_TO_AOI = (
    "projects/thurgau-irrigation/assets/Zuerich/Zuerich_bound"
)

aoi_feature_collection = ee.FeatureCollection(PATH_TO_AOI)
aoi_geometry = aoi_feature_collection.geometry().simplify(500)

AOI = aoi_geometry.buffer(100)

In [ ]:
def process_and_export_et_green(
    et_collection_list: ee.List,
    year: int,
    aoi: ee.Geometry,
    time_steps: int = 36,
    time_step_type: str = "dekadal",
    double_cropping_image: ee.Image = None,
) -> List[ee.batch.Task]:
    """
    Process and export ET green images for a given year.

    Args:
        et_collection (ee.List): List of ET images
        year (int): Year to process
        aoi (ee.Geometry): Area of interest
        time_steps (int): Number of time steps (36 for dekadal, 12 for monthly)
        time_step_type (str): Type of time step ("dekadal" or "monthly")
        double_cropping_image (ee.Image): Double cropping image

    Returns:
        List[ee.batch.Task]: List of export tasks
    """
    # Load collections for the year
    nutzung_collection = ee.FeatureCollection(
        f"projects/thurgau-irrigation/assets/ZH_Nutzungsflaechen_2/ZH_Nutzungsflaechen_{year}"
    )

    jurisdictions = ee.FeatureCollection(
        f"projects/thurgau-irrigation/assets/Zuerich/zurich_jurisdictions_{year}_wgs84"
    )

    double_cropping = ee.Image(double_cropping_image)

    # Prepare crop filters
    not_irrigated_crops = get_crops_to_exclude()
    rainfed_crops = get_rainfed_reference_crops()
    exclude_filter, rainfed_filter = create_crop_filters(
        not_irrigated_crops, rainfed_crops
    )

    # Process nutzung collection
    nutzung_with_double_crop = add_double_cropping_info(
        nutzung_collection, double_cropping
    )
    _, rainfed_fields = filter_crops(
        nutzung_with_double_crop, exclude_filter, rainfed_filter
    )

    tasks = []
    for i in range(time_steps):
        # Get time step info
        if time_step_type == "dekadal":
            dekadal = i % 3 + 1
            month = i // 3 + 1
            time_step_name = f"{month:02d}_D{dekadal}"
        elif time_step_type == "monthly":
            month = i + 1
            time_step_name = f"{month:02d}"
        else:
            raise ValueError("time_step_type must be either 'dekadal' or 'monthly'")

        # Get ET image and compute ET green
        et_image = ee.Image(et_collection_list.get(i))
        et_green = compute_et_green(et_image, rainfed_fields, jurisdictions)

        # Export result
        task_name = (
            f"ET_green_wapor_jurisdiction_zh_{time_step_type}_{year}-{time_step_name}"
        )
        asset_id = f"projects/thurgau-irrigation/assets/Zuerich/ET_green_wapor_jurisdiction_zh_{time_step_type}_2022/{task_name}"

        task = export_image_to_asset(et_green, asset_id, task_name, year, aoi)
        tasks.append(task)

    return tasks

In [ ]:
for year in YEARS:

    
    tasks = process_and_export_et_green(
        et_collection_list, year, AOI, TIME_STEPS, TIME_STEP_TYPE, DOUBLE_CROPPING_IMAGE
    )

    print(f"Started {len(tasks)} export tasks.")